In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-16:] == 'image-to-tex-OCR':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR/Jupyter_Notebooks
Current path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR


In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
from Data.Data_Module_CFG import Data_Module_CFG
from Models.Printed_Tex_Transformer import ResNetTransformer
from Lightning_Models.Printed_Tex_Lit_Model import LitResNetTransformer
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from PIL import Image
import torch
from pytorch_lightning.loggers import TensorBoardLogger
from torchvision import transforms
from PIL import Image
from IPython.display import display, Math
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")
torch.cuda.is_available()

False

In [4]:


class CFG:
    
    device = dev
    
   
    use_wandb = True
    
    ######### Dataset #########
    
    # stage: 'train' or 'test'
    stage = 'train'
    set_max_label_length = 128
    number_png_images_to_use_in_dataset=200
    labels_transform='default'
    image_transform_name ='alb'
    train_val_fraction = 0.9
    vocabulary_path = 'Tokenizer/230k_ver2.json'
    load_vocabulary = True
    image_padding = True
    max_width = 10*1000


    batch_size = 8
    num_workers = 0
    on_gpu = False
    
    ######## Model and Lightning Model paramters ############
    
    # MODEL
    
    
    
    
    
    checkpoint = None
    save_directory = None
    
    
    accumulate_grad_batches = 64 // batch_size  # experiments showed batch_size * accumulate_grad = 192 is optimal
    learning_rate =  1e-5
    eta_min = 1e-8
    t_max = 40
    max_epochs = 35
    weight_decay =  1e-5
    precision =16
    
    # checkpointing
    save_top_k=5
    
    monitor="FBETA"
    mode="max"
    
    
    ####### Augemtnations ###############
    
    # Training Aug
    train_transforms = [
        # A.RandomResizedCrop(
        #     size, size, scale=(0.85, 1.0)),
        #A.Resize(patch_size, patch_size),
          A.augmentations.geometric.resize.Resize(interpolation= cv2.INTER_LINEAR,height=128, width= 128, p=1),
        
     
        ToTensorV2(),
    ]
    

    
    # Validaiton Aug
    val_transforms = [
        #A.Resize(patch_size, patch_size),
    A.augmentations.geometric.resize.Resize(interpolation= cv2.INTER_LINEAR,height=64, width= 256, p=1),

       
        ToTensorV2(),
    ]
    
    # Test Aug
    test_transforms = [
        #A.Resize(patch_size, patch_size),
      

        ToTensorV2(),
    ]
        
    
    

In [5]:
# Generate Data Module by uploading images and formulas
# images need to be in the folder Data/Data_Bank/generated_png_images
# formulas need to be in Data/Data_Bank/final_png_formulas.txt
# image filenames need to be in Data/Data_Bank/corresponding_png_images.txt

dataset = Data_Module_CFG(cfg=CFG
                )


# Generates DataFrame and vocabulary, tokezniers etc
#dataset.prepare_data()

# prepeares the dataloaders with transformations and splits train/val
#dataset.setup(stage = 'train')

Train/Val Data is ready for Model loading.


In [6]:
print( 
    'Vocabulary size:',len(dataset.vocabulary),
    "\nTrain-Dataset size:", len(dataset.data_train),
    "\nValidation-Dataset size:", len(dataset.data_val),
    '\nMax label length:', dataset.max_label_length,
    "\nStart <S> goes to index ",dataset.vocabulary['<S>'],
      "\nEnd <E> goes to index ",dataset.vocabulary['<E>'],
      "\nPadding <P> goes to index ",dataset.vocabulary['<P>'],
)

Vocabulary size: 581 
Train-Dataset size: 184 
Validation-Dataset size: 27 
Max label length: 130 
Start <S> goes to index  0 
End <E> goes to index  1 
Padding <P> goes to index  2


[89,
 59,
 35,
 12,
 60,
 32,
 121,
 123,
 198,
 169,
 72,
 3,
 124,
 108,
 19,
 188,
 189,
 63,
 173,
 33,
 161,
 101,
 102,
 127,
 57,
 105,
 95,
 145,
 38,
 157,
 126,
 98,
 187,
 139,
 183,
 197,
 7,
 53,
 90,
 81,
 155,
 58,
 14,
 93,
 88,
 177,
 107,
 96,
 79,
 74,
 135,
 40,
 9,
 61,
 168,
 140,
 115,
 65,
 5,
 106,
 42,
 36,
 1,
 148,
 162,
 11,
 178,
 21,
 116,
 52,
 190,
 146,
 69,
 48,
 30,
 191,
 62,
 22,
 193,
 180,
 67,
 75,
 165,
 83,
 119,
 18,
 87,
 27,
 179,
 82,
 92,
 16,
 17,
 6,
 28,
 51,
 174,
 2,
 56,
 55,
 128,
 111,
 184,
 50,
 34,
 66,
 172,
 94,
 131,
 84,
 195,
 26,
 91,
 141,
 0,
 110,
 156,
 23,
 134,
 138,
 31,
 10,
 24,
 8,
 112,
 77,
 142,
 170,
 137,
 132,
 43,
 120,
 39,
 20,
 118,
 86,
 73,
 41,
 45,
 64,
 176,
 114,
 47,
 122,
 154,
 158,
 152,
 182,
 181,
 199,
 78,
 136,
 37,
 46,
 113,
 133,
 160,
 153,
 99,
 150,
 29,
 49,
 143,
 109,
 70,
 100,
 185,
 104,
 44,
 125,
 85,
 171,
 103,
 147,
 117,
 164,
 175,
 163,
 129,
 167,
 192]

,formula,image_name,height,width,formula_length,aspect_ratio,latex_tokenized,tokenized_len
0,\gamma _ { C } ( g _ { \mathrm { R } } ) = - \...,8c2ef650d0104e0.png,128,515,151,4,"[\gamma, _, {, C, }, (, g, _, {, \mathrm, {, R...",58
1,G ( q ^ { 2 } ) = i \frac { \alpha h - g } { \...,283d9468d1d7ebc.png,128,483,111,3,"[G, (, q, ^, {, 2, }, ), =, i, \frac, {, \alph...",40
2,w ^ { a b } = w ^ { a b } ( e ) + \kappa ^ { a...,f333d8cf4a4c201.png,128,909,56,7,"[w, ^, {, a, b, }, =, w, ^, {, a, b, }, (, e, ...",26
3,E = \frac { 1 } { 4 G _ { 4 } } \sum _ { i } Q...,a03f7b7a61e5c89.png,128,687,134,5,"[E, =, \frac, {, 1, }, {, 4, G, _, {, 4, }, },...",58
4,\left( v _ { \xi } \right) _ { \ \alpha } ^ { ...,9631cf277eca635.png,128,536,98,4,"[\left(, v, _, {, \xi, }, \right), _, {, \, \a...",29
5,"{ \cal L } _ { \mathrm { F } } = i \, \display...",52307f2cc63996e.png,128,963,201,7,"[{, \cal, L, }, _, {, \mathrm, {, F, }, }, =, ...",71
6,\partial _ { - } ( e ^ { \chi f - 2 \rho } \Om...,e608cd96fc31065.png,128,1152,105,9,"[\partial, _, {, -, }, (, e, ^, {, \chi, f, -,...",38
7,"\{ E ( Q ) , G ( Q ) \} = { \frac { { \partial...",08e06d046c86c49.png,128,784,182,6,"[\{, E, (, Q, ), ,, G, (, Q, ), \}, =, {, \fra...",71
8,I ( \gamma _ { k } ) = \int _ { \gamma _ { k }...,e3e08641a38faf2.png,128,418,74,3,"[I, (, \gamma, _, {, k, }, ), =, \int, _, {, \...",28
10,S = - \int d ^ { N } x \sqrt { g } e ^ { - 2 \...,6fd3dc5a762406f.png,128,1172,200,9,"[S, =, -, \int, d, ^, {, N, }, x, \sqrt, {, g,...",62


In [11]:
dataset.data_train.dataframe

,formula,image_name,height,width,formula_length,aspect_ratio,latex_tokenized,tokenized_len
0,\gamma _ { C } ( g _ { \mathrm { R } } ) = - \...,8c2ef650d0104e0.png,128,515,151,4,"[\gamma, _, {, C, }, (, g, _, {, \mathrm, {, R...",58
1,G ( q ^ { 2 } ) = i \frac { \alpha h - g } { \...,283d9468d1d7ebc.png,128,483,111,3,"[G, (, q, ^, {, 2, }, ), =, i, \frac, {, \alph...",40
2,w ^ { a b } = w ^ { a b } ( e ) + \kappa ^ { a...,f333d8cf4a4c201.png,128,909,56,7,"[w, ^, {, a, b, }, =, w, ^, {, a, b, }, (, e, ...",26
3,E = \frac { 1 } { 4 G _ { 4 } } \sum _ { i } Q...,a03f7b7a61e5c89.png,128,687,134,5,"[E, =, \frac, {, 1, }, {, 4, G, _, {, 4, }, },...",58
5,"{ \cal L } _ { \mathrm { F } } = i \, \display...",52307f2cc63996e.png,128,963,201,7,"[{, \cal, L, }, _, {, \mathrm, {, F, }, }, =, ...",71
...,...,...,...,...,...,...,...,...
206,u ^ { \alpha } = W ^ { - 1 } \widetilde { u } ...,60e1c0a4612d9cd.png,128,1223,106,9,"[u, ^, {, \alpha, }, =, W, ^, {, -, 1, }, \wid...",34
207,L = \sum _ { i = 1 } ^ { N } [ { \frac { 1 } {...,674bab8e54384d4.png,128,954,202,7,"[L, =, \sum, _, {, i, =, 1, }, ^, {, N, }, [, ...",94
208,"4 m \kappa ^ { 2 } \, Z _ { m } ^ { \prime \pr...",8fdb7e6f7cf0204.png,128,1449,192,11,"[4, m, \kappa, ^, {, 2, }, \,, Z, _, {, m, }, ...",72
209,\zeta ( s ) = \int { \frac { d ^ { 4 } p } { (...,52914d8e392cfab.png,128,1046,248,8,"[\zeta, (, s, ), =, \int, {, \frac, {, d, ^, {...",108


In [16]:
len(dataset.data_.image_filenames)

27

In [ ]:
# Helper Function to convert prediction labels to strings
def token_to_strings(tokens):
    mapping = dataset.vocabulary
    inverse_mapping =dataset.inverse_vocabulary
    s=''
    if tokens.shape[0] ==1:
        tokens = tokens[0]
    for number in tokens:
        letter = inverse_mapping[number.item()]
        s= s + str(letter)
    return s





In [ ]:

transform = transforms.ToPILImage()
display(transform(dataset.data_train[1][0]))
print(token_to_strings(dataset.data_train[10][1]))

In [ ]:
transform = transforms.ToPILImage()
display(transform(dataset.data_val[10][0]))
print(token_to_strings(dataset.data_val[10][1]))

In [ ]:
print q

In [ ]:
# Initiate the model
model = ResNetTransformer(dataset=dataset).to(dev)

In [ ]:
torch.set_num_threads(1)
logger = TensorBoardLogger("Models_Parameters_Log", name="Test_Run")
lit_model = LitResNetTransformer(model=model)

#use the lit_model to run with Trainer
trainer = Trainer(accelerator="mps",   max_epochs=5, logger=logger,) #accumulate_grad_batches=2, overfit_batches=1, default_root_dir="lightning_logs/ResNet/checkpoints"  
trainer.fit(model=lit_model, datamodule=dataset,)

#add parameter below to trainer.fit to continue from a checkpoint
#ckpt_path='lightning_logs/Character_tokenizer_1/version_2/checkpoints/epoch=0-step=1294.ckpt'
 #ckpt_path='lightning_logs/Character_tokenizer_1/version_4/checkpoints/epoch=15-step=20704.ckpt' )

In [ ]:
import json

def save_dic(dic):
    filename = 'lightning_logs/256_character_1.json'
    with open(filename, 'w') as f:
        f.write(json.dumps(dic, default=str))

def load_dic(filename):
    with open(filename) as f:
        dic = json.loads(f.read())
    return dic

In [ ]:
# Helper Function to convert prediction labels to strings
def token_to_strings(tokens):
    mapping = dataset.vocabulary
    inverse_mapping =dataset.inverse_vocabulary
    s=''
    if tokens.shape[0] ==1:
        tokens = tokens[0]
    for number in tokens:
        letter = inverse_mapping[number.item()]
        s= s +" " + str(letter)
    return s


In [ ]:
# Get image and label from train data -- change number for different ones
image_tensor, label = dataset.data_train[12]
transform = transforms.ToPILImage()
image = transform(image_tensor)

print('\nOriginal image and formula:')
display(image)
print(token_to_strings(label))
print('\nPredicted formula: \n')
with torch.no_grad():
    model.to('cpu')
    model.eval()
    prediction =  model.predict(image_tensor.unsqueeze(0).to('cpu'))
print(token_to_strings(prediction),'\n')
print(display(Math(token_to_strings(prediction))))

In [ ]:
data